In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List, Annotated
from IPython.display import Image, display 
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
import operator

In [2]:
essay = """The rapid growth of technology has transformed nearly every aspect of human life. From communication to transportation, innovation continues to push boundaries and redefine possibilities. Artificial intelligence, for instance, has not only automated repetitive tasks but also enhanced creativity and decision-making across industries. However, this progress also brings challenges, such as ethical dilemmas, privacy concerns, and the risk of overdependence on machines. To truly benefit from technological advancement, society must strike a balance between innovation and responsibility. Education, awareness, and thoughtful regulation can ensure that technology remains a tool for empowerment rather than a force of disruption."""

In [ ]:
"""
Trying to get structured output

# 1. Defining the output schema
class output_schema(BaseModel):
    feedback : str = Field(description="This stores the feedback from the llm")
    score : int = Field(description="This stores the score of the essay")

# 2. Initializing the model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash").with_structured_output(output_schema)

# 3. Creating the Prompt
prompt = PromptTemplate.from_template("Provide a detailed feedback and a score to the following essay from 0-10\n\n essay = {essay}")

# 4. Invoking the model
response = llm.invoke(prompt.format(essay=essay))

print(response.feedback)
print(response.score)

"""

The essay provides a concise yet comprehensive overview of the impact of technology. It effectively highlights both the transformative benefits, such as AI's role in automation and enhanced decision-making, and the critical challenges, including ethical dilemmas and privacy concerns. The structure is clear, moving logically from impact to challenges and then offering thoughtful solutions like education and regulation. The language used is precise and professional. To further strengthen the argument, you could consider providing more specific examples for each point, such as concrete instances of AI enhancing creativity or particular ethical dilemmas that arise. Overall, it's a well-articulated piece that demonstrates a strong understanding of the topic.
8


In [5]:
class output_schema(BaseModel):
    feedback : str = Field(description="This stores the feedback from the llm")
    score : int = Field(description="This stores the score of the essay")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
structured_llm = llm.with_structured_output(output_schema)


In [3]:
class UPSCstate(TypedDict):
    essay_text : str
    cot_feedback : str
    doa_feedback : str
    lang_feedback : str
    scores : Annotated[List[int], operator.add]
    final_feedback : str
    final_score : int


In [6]:
def cot_eval(state : UPSCstate) -> UPSCstate:
    prompt = PromptTemplate.from_template("Provide detailed feedback and a score to the following essay from 0-10 focusing on the chain of thought.\n\n essay = {essay}")
    response = structured_llm.invoke(prompt.format(essay=state["essay_text"]))
    return {'cot_feedback': response.feedback, 'scores': [response.score]}

In [7]:
def doa_eval(state : UPSCstate) -> UPSCstate:
    prompt = PromptTemplate.from_template("Provide detailed feedback and a score to the following essay from 0-10 focusing on depth of analysis.\n\n essay = {essay}")
    response = structured_llm.invoke(prompt.format(essay=state["essay_text"]))
    return {'doa_feedback': response.feedback, 'scores': [response.score]}

In [8]:
def lang_eval(state : UPSCstate) -> UPSCstate:
    prompt = PromptTemplate.from_template("Provide detailed feedback and a score to the following essay from 0-10 focusing on language and grammar.\n\n essay = {essay}")
    response = structured_llm.invoke(prompt.format(essay=state["essay_text"]))
    return {'lang_feedback': response.feedback, 'scores': [response.score]}

In [14]:
def final_eval(state : UPSCstate) -> UPSCstate:
    prompt = PromptTemplate.from_template("Provide a consolidated feedback based on the following feedbacks. \n\n chain of thought feedback = {cot_feedback}\n depth of analysis feedback = {doa_feedback}\n language feedback = {lang_feedback}\n\n")
    response = llm.invoke(prompt.format(cot_feedback=state["cot_feedback"], doa_feedback=state["doa_feedback"], lang_feedback=state["lang_feedback"])).content
    avg_score = sum(state["scores"]) // len(state["scores"])

    return {'final_feedback': response, 'final_score': avg_score}

In [15]:
graph = StateGraph(UPSCstate)

graph.add_node("cot_eval", cot_eval)
graph.add_node("doa_eval", doa_eval)
graph.add_node("lang_eval", lang_eval)
graph.add_node("final_eval", final_eval)

graph.add_edge(START, "cot_eval")
graph.add_edge(START, "doa_eval")
graph.add_edge(START, "lang_eval")
graph.add_edge("cot_eval", "final_eval")
graph.add_edge("doa_eval", "final_eval")
graph.add_edge("lang_eval", "final_eval")
graph.add_edge("final_eval", END)

workflow = graph.compile()


In [16]:
initial_state = {'essay_text': essay}
final_state = workflow.invoke(initial_state)

In [17]:
print(final_state["final_feedback"])

Here's a consolidated feedback based on the provided points:

Overall, the essay demonstrates a strong foundation in structure and language, making it highly readable and persuasive.

**Strengths:**

*   **Exceptional Chain of Thought:** The essay exhibits a very clear, coherent, and logical progression of ideas. It smoothly transitions from a broad overview of technology's impact to specific examples like AI, then effectively explores both its advantages and challenges, and finally proposes well-reasoned solutions. The argument flows naturally, creates a balanced perspective, and is easy to follow, showcasing a strong understanding of how to construct an informative and persuasive piece.
*   **Excellent Command of Language:** The writing is polished, professional, and grammatically flawless. Sentences are well-structured, clear, and concise, with precise and appropriate vocabulary. The effective use of transition words ensures a smooth flow between ideas, contributing significantly to

In [18]:
print(final_state["final_score"])

8


In [19]:
print(final_state["scores"])

[9, 7, 9]
